In [ ]:
import subprocess
from pydub import AudioSegment
import math

def extract_audio_from_video(video_path, audio_path):
    command = [
            "ffmpeg",
            "-i", 
            video_path, 
            "-vn", audio_path
        ]
    subprocess.run(command)
def cut_audio_in_chunks(audio_path, chunk_size, chunks_dir):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000

    track = AudioSegment.from_mp3("./files/podcast.mp3")

    chunks = math.ceil(len(track)/chunk_len)

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len

        chunk = track[start_time:end_time]

        chunk.export(f"{chunks_dir}/chunk_{i}.mp3", format="mp3")

In [ ]:
cut_audio_in_chunks("./files/podcast.mp3", 10, "./files/chunks")

In [ ]:
import openai
import glob

# transcript["text"]

def transcribe_chunks(chunk_dir, destination):
    files = glob.glob(f"{chunk_dir}/*.mp3")
    fianl_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file
            )
            fianl_transcript += transcript["text"]
    with open(destination, "w") as file:
        file.write(fianl_transcript)
   

transcribe_chunks("./files/chunks", "./files/transcript.txt")